In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gtts import gTTS
import IPython.display as ipd
from deep_translator import GoogleTranslator

# Load the dataset
df = pd.read_csv('alzheimers_chatbot_data.csv')  # Update with your CSV file path

# Preprocess the text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    return text

# Preprocess the questions in the dataset
df['Preprocessed_Question'] = df['Question'].apply(preprocess_text)

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['Preprocessed_Question'])

# Function to find the best match for a user query and return the relevance score
def get_best_answer(user_query):
    user_query = preprocess_text(user_query)
    query_vector = vectorizer.transform([user_query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    best_match_index = cosine_similarities.argmax()  # Get index of the highest similarity
    best_answer = df.iloc[best_match_index]['Answer']
    relevance_score = cosine_similarities[best_match_index]  # Get relevance score
    return best_answer, relevance_score

# Function to convert text to speech using gTTS
def speak(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    audio_file = 'output.mp3'
    tts.save(audio_file)
    return audio_file


# Main loop for user interaction
print("Welcome to the Alzheimer's Chatbot! Type 'exit' to quit.")

while True:
    lang_choice = input("Which language would you like to hear the answer in? (english/hindi/spanish/french): ").strip().lower()
    if lang_choice == 'hindi':
        target_lang = 'hi'
    elif lang_choice == 'spanish':
        target_lang = 'es'
    elif lang_choice == 'french':
        target_lang = 'fr'
    elif lang_choice =='english' : 
        target_lang ='en'
        
    user_query = input("You: ")
    if user_query.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break

    answer, relevance_score = get_best_answer(user_query)
    print(f"Chatbot: {answer}")
    print(f"Relevance Score: {relevance_score:.2f}")  # Display relevance score

    # Ask if the user wants to hear the answer
    hear_answer = input("Would you like to hear the answer? (yes/no): ").strip().lower()
    if hear_answer == 'yes':
        # Ask for the language preference
        translated_answer = GoogleTranslator(source='en', target= target_lang).translate(answer)
        audio_file = speak(translated_answer, lang= target_lang)  # Generate audio file in Hindi
        import pygame

        # Initialize pygame mixer
        pygame.mixer.init()

        # Load the sound file
        sound = pygame.mixer.Sound('output.mp3')

        # Play the sound
        sound.play()

        # Wait for the sound to finish playing
        pygame.time.delay(int(sound.get_length() * 1000))

        # Quit pygame mixer
        pygame.mixer.quit()

Welcome to the Alzheimer's Chatbot! Type 'exit' to quit.
Chatbot: Alzheimer's disease is a progressive brain disorder that leads to memory loss, confusion, and cognitive decline.
Relevance Score: 0.00
